# App Search Engine exporter to Elasticsearch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/enterprise-search/app-search-engine-exporter.ipynb)

This notebook explains the steps of exporting an App Search engine together with its configurations in Elasticsearch. This is not meant to be an exhaustive example for all App Search features as those will vary based on your instance, but is meant to give a sense of how you can export, migrate, and enhance your application.

We will look at:

- [how to export synonyms](#export-app-search-synonyms-in-elasticsearch)
- [how to export curations](#export-app-search-curations-in-elasticsearch)
- [how to create a new index in Elasticsearch](#create-a-new-elasticsearch-index)
- [how to add sparse vector fields](#add-sparse_vector-fields-for-semantic-search-optional)
- [how to query the new Elasticsearch index](#query-the-new-elasticsearch-index)

## Setup

Let's start by making sure our Elasticsearch and Enterprise Search clients are installed. We'll also use `getpass` to ensure we can allow secure user inputs for our IDs and keys to access our Elasticsearch instance.


In [ ]:
# install packages
import sys

!{sys.executable} -m pip install -qU elasticsearch elastic-enterprise-search

# import modules
from getpass import getpass
from elastic_enterprise_search import AppSearch
from elasticsearch import Elasticsearch
import json

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment. 

You will also need your **API KEY** to access your deployment. You can [create a new API key](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key) from the `Stack Management -> API keys` menu in Kibana. Be sure to copy or write down your key in a safe place once it is created it will be displayed only upon creation.


In [ ]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

elasticsearch = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

## Connect to App Search

For this notebook we will need access to an App Search private key that can access the App Search engine we want to export.
We will be instantiating the Enterprise Search client with the provided credentials and then check that we are correctly authenticated to Enterprise Search by getting the App Search engine details.

You can find your App Search endpoint and your search private key from the `Credentials` menu inside your App Search instance in Kibana.

Also note here, we define our `ENGINE_NAME`. For this examplem we are using the `national-parks-demo` sample engine that is available within App Search.

In [ ]:
APP_SEARCH_ENDPOINT = getpass("App Search endpoint: ")
APP_SEARCH_PRIVATE_KEY = getpass("App Search private key: ")

app_search = AppSearch(APP_SEARCH_ENDPOINT, bearer_auth=APP_SEARCH_PRIVATE_KEY)

# modify this with your own engine name
ENGINE_NAME = "national-parks-demo"

## Export App Search synonyms in Elasticsearch

To get started with our export, we will first export any [synonyms](https://www.elastic.co/guide/en/app-search/current/synonyms-guide.html) we have in our App Search engine. 

The resulting synonyms will be placed into a new [Elasticsearch synoynm set](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-with-synonyms.html) named the same as our App Search egnine and used in analyzers for our synonyms-filter type later on.



In [ ]:
elasticsearch.synonyms.put_synonym(id=ENGINE_NAME, synonyms_set=[])

for synonym_set in app_search.list_synonym_sets(engine_name=ENGINE_NAME).body[
    "results"
]:
    elasticsearch.synonyms.put_synonym_rule(
        set_id=ENGINE_NAME,
        rule_id=synonym_set["id"],
        synonyms=", ".join(synonym_set["synonyms"]),
    )

## Export App Search curations in Elasticsearch

Next, we will export any curations that may be in our App Search engine.

To export App Search curations we will use Elasticsearch [query rules](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-using-query-rules.html).
At the moment of writing this notebook Elasticsearch query rules only allow for pinning results unlike App Search curations that also allow excluding results.
For this reason we will only export pinned results. The code below will create the necessary `query_rules` to achieve this. Note that there is a default soft limit of 100 curations for `query_rules` that can be configured up to a hard limit of 1,000.

In [ ]:
query_rules = []

for curation in app_search.list_curations(engine_name=ENGINE_NAME).body["results"]:
    query_rules.append(
        {
            "rule_id": curation["id"],
            "type": "pinned",
            "criteria": [
                {
                    "type": "exact",
                    "metadata": "user_query",
                    "values": curation["queries"],
                }
            ],
            "actions": {"ids": curation["promoted"]},
        }
    )


elasticsearch.query_ruleset.put(ruleset_id=ENGINE_NAME, rules=query_rules)

## Create a new Elasticsearch index

While we could re-use the same Elasticsearch index that is storing the App Search engine documents, reindexing the data in a new index will allow us to change the mapping to use features like semantic search or to be able to use the Elasticsearch synonym set we just created.

App Search has the following data types: text, number, date and geolocation. Each of these types is mapped to Elasticsearch field types.
We can take a closer look at how App Search field types are mapped to Elasticsearch fields, by using the [`GET mapping API`](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-get-mapping.html).
For App Search engines, the associated Elasticsearch index name is `.ent-search-engine-documents-[ENGINE_NAME]`, e.g. `.ent-search-engine-documents-national-parks-demo` for the App Search sample engine `national-parks-demo`.
One thing to notice is how App Search uses [multi-fields](https://www.elastic.co/guide/en/elasticsearch/reference/current/multi-fields.html) in Elasticsearch that allow for quickly changing the field type in App Search without requiring reindexing by creating subfields for each type of supported field:

<details>
  <summary>An example schema can be found by clicking here</summary>

```json
"[APP_SEARCH_ENGINE_FIELD_NAME]": {
  "type": "text",
  "fields": {
    "date": {
      "type": "date",
      "format": "strict_date_time||strict_date",
      "ignore_malformed": true
    },
    "delimiter": {
      "type": "text",
      "index_options": "freqs",
      "analyzer": "iq_text_delimiter"
    },
    "enum": {
      "type": "keyword",
      "ignore_above": 2048
    },
    "float": {
      "type": "double",
      "ignore_malformed": true
    },
    "joined": {
      "type": "text",
      "index_options": "freqs",
      "analyzer": "i_text_bigram",
      "search_analyzer": "q_text_bigram"
    },
    "location": {
      "type": "geo_point",
      "ignore_malformed": true,
      "ignore_z_value": false
    },
    "prefix": {
      "type": "text",
      "index_options": "docs",
      "analyzer": "i_prefix",
      "search_analyzer": "q_prefix"
    },
    "stem": {
      "type": "text",
      "analyzer": "iq_text_stem"
    }
  },
  "index_options": "freqs",
  "analyzer": "iq_text_base"
}
```
</details>

In our case we can assume that we have a well established schema and we do not need to use all multi-fields.

We can retrieve the field types of an App Search engine using the [Schema API](https://www.elastic.co/guide/en/app-search/current/schema.html) and then construct our mapping.

Also note that below, we set up variables for our `SOURCE_INDEX` and `DEST_INDEX`. If you want your destination index to be named differently, you can edit it here as these variables are used throughout the rest of the notebook.

In [ ]:
# define SOURCE_INDEX and DEST_INDEX which we will continue to reuse; feel free to adjust DEST_INDEX
SOURCE_INDEX = ".ent-search-engine-documents-" + ENGINE_NAME
DEST_INDEX = "new-" + ENGINE_NAME

# delete the index if it's already created
if elasticsearch.indices.exists(index=DEST_INDEX):
    elasticsearch.indices.delete(index=DEST_INDEX)

# get the App Search engine schema
schema = app_search.get_schema(engine_name=ENGINE_NAME)

# construct the Elasticsearch mapping
mapping = {}

for field_name in schema:
    field_type = schema[field_name]

    if field_type == "date":
        mapping[field_name] = {
            "type": "date",
            "format": "strict_date_time||strict_date",
            "ignore_malformed": True,
        }
    elif field_type == "location":
        mapping[field_name] = {"type": "geo_point", "ignore_z_value": False}
    elif field_type == "number":
        mapping[field_name] = {"type": "double"}
    elif field_type == "text":
        # feel free to modify this with your own mapping for text fields
        mapping[field_name] = {
            "fields": {
                "keyword": {"type": "keyword", "ignore_above": 2048},
                "delimiter": {
                    "type": "text",
                    "index_options": "freqs",
                    "analyzer": "iq_text_delimiter",
                },
                "joined": {
                    "type": "text",
                    "index_options": "freqs",
                    "analyzer": "i_text_bigram",
                    "search_analyzer": "q_text_bigram",
                },
                "prefix": {
                    "type": "text",
                    "index_options": "docs",
                    "analyzer": "i_prefix",
                    "search_analyzer": "q_prefix",
                },
                "stem": {"type": "text", "analyzer": "iq_text_stem"},
            },
            "type": "text",
            "index_options": "freqs",
            "analyzer": "i_text_base",
            "search_analyzer": "q_text_base",
        }

# These are similar to the Elasticsearch analyzers we use for App Search.
# The main difference is that we are also adding a synonyms filter so that we can
# leverage the Elasticsearch synonym set we created in a previous step.
# If you want a different mapping for text fields, feel free to modify.
settings = {
    "analysis": {
        "filter": {
            "front_ngram": {"type": "edge_ngram", "min_gram": "1", "max_gram": "12"},
            "bigram_joiner": {
                "max_shingle_size": "2",
                "token_separator": "",
                "output_unigrams": "false",
                "type": "shingle",
            },
            "bigram_max_size": {"type": "length", "max": "16", "min": "0"},
            "en-stem-filter": {"name": "light_english", "type": "stemmer"},
            "bigram_joiner_unigrams": {
                "max_shingle_size": "2",
                "token_separator": "",
                "output_unigrams": "true",
                "type": "shingle",
            },
            "delimiter": {
                "split_on_numerics": "true",
                "generate_word_parts": "true",
                "preserve_original": "false",
                "catenate_words": "true",
                "generate_number_parts": "true",
                "catenate_all": "true",
                "split_on_case_change": "true",
                "type": "word_delimiter_graph",
                "catenate_numbers": "true",
                "stem_english_possessive": "true",
            },
            "en-stop-words-filter": {"type": "stop", "stopwords": "_english_"},
            "synonyms-filter": {
                "type": "synonym_graph",
                "synonyms_set": ENGINE_NAME,
                "updateable": True,
            },
        },
        "analyzer": {
            "i_prefix": {
                "filter": ["cjk_width", "lowercase", "asciifolding", "front_ngram"],
                "tokenizer": "standard",
            },
            "iq_text_delimiter": {
                "filter": [
                    "delimiter",
                    "cjk_width",
                    "lowercase",
                    "asciifolding",
                    "en-stop-words-filter",
                    "en-stem-filter",
                ],
                "tokenizer": "whitespace",
            },
            "q_prefix": {
                "filter": ["cjk_width", "lowercase", "asciifolding"],
                "tokenizer": "standard",
            },
            "i_text_base": {
                "filter": [
                    "cjk_width",
                    "lowercase",
                    "asciifolding",
                    "en-stop-words-filter",
                ],
                "tokenizer": "standard",
            },
            "q_text_base": {
                "filter": [
                    "cjk_width",
                    "lowercase",
                    "asciifolding",
                    "en-stop-words-filter",
                    "synonyms-filter",
                ],
                "tokenizer": "standard",
            },
            "iq_text_stem": {
                "filter": [
                    "cjk_width",
                    "lowercase",
                    "asciifolding",
                    "en-stop-words-filter",
                    "en-stem-filter",
                ],
                "tokenizer": "standard",
            },
            "i_text_bigram": {
                "filter": [
                    "cjk_width",
                    "lowercase",
                    "asciifolding",
                    "en-stem-filter",
                    "bigram_joiner",
                    "bigram_max_size",
                ],
                "tokenizer": "standard",
            },
            "q_text_bigram": {
                "filter": [
                    "cjk_width",
                    "lowercase",
                    "asciifolding",
                    "synonyms-filter",
                    "en-stem-filter",
                    "bigram_joiner_unigrams",
                    "bigram_max_size",
                ],
                "tokenizer": "standard",
            },
        },
    }
}

# and actually create our index
elasticsearch.indices.create(
    index=DEST_INDEX, mappings={"properties": mapping}, settings=settings
)

# Add `sparse_vector` fields for semantic search (optional)

One of the advantages of having our exported index directly in Elasticsearch is that we can easily take advantage of doing semantic search with ELSER. To do this, we'll need to add a `sparse_vector` field to our index, set up an ingest pipeline, and reindex our data.

Note that to use this feature, your cluster must have at least one ML node set up with enough resources allocated to it.

Let's first start by adding `sparse_vector` fields to our new index mapping.

In [ ]:
# by default we are adding a `sparse_vector` field for all text fields in our engine
# feel free to modify this list to only include the fields that are relevant
SPARSE_VECTOR_FIELDS = [
    field_name + "_semantic" for field_name in schema if schema[field_name] == "text"
]

sparse_vector_fields = {}
for field_name in SPARSE_VECTOR_FIELDS:
    # this is added so we can use semantic search with ELSER
    sparse_vector_fields[field_name] = {"type": "sparse_vector"}

elasticsearch.indices.put_mapping(index=DEST_INDEX, properties=sparse_vector_fields)

## Setup an ingest pipeline using ELSER

> If you have not already deployed ELSER, follow this [guide](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html) on how to download and deploy the model. Without this step, you will receive errors below when you run the `reindex` command.

Assuming you have downloaded and deployed ELSER in your deployment, we can now define an ingest pipeline that will enrich the documents with the `sparse_vector` fields that can be used with semantic search.

In [ ]:
PIPELINE = "elser-ingest-pipeline-" + ENGINE_NAME

processors = []

for output_field in SPARSE_VECTOR_FIELDS:
    input_field = output_field.removesuffix("_semantic")
    processors.append(
        {
            "inference": {
                "model_id": ".elser_model_2",
                "input_output": [
                    {"input_field": input_field, "output_field": output_field}
                ],
                "on_failure": [
                    {
                        "append": {
                            "field": "_source._ingest.inference_errors",
                            "allow_duplicates": False,
                            "value": [
                                {
                                    "message": "Processor failed for field '"
                                    + input_field
                                    + "' with message '{{ _ingest.on_failure_message }}'",
                                    "timestamp": "{{{ _ingest.timestamp }}}",
                                }
                            ],
                        }
                    }
                ],
            }
        }
    )

# create the ingest pipeline
elasticsearch.ingest.put_pipeline(
    id=PIPELINE, description="Ingest pipeline for ELSER", processors=processors
)

## Reindex the data
Now that we have created the Elasticsearch index and the ingest pipeline, it's time to reindex our data in the new index. The pipeline definition we created above will create a field for each of the `SPARSE_VECTOR_FIELDS` we defined with a `_semantic` suffix, and then infer the sparse vector values from ELSER as the reindex takes place.

In [ ]:
reindex_task = elasticsearch.reindex(
    source={"index": SOURCE_INDEX},
    dest={"index": DEST_INDEX, "pipeline": PIPELINE},
    wait_for_completion=False,
)

task_id = reindex_task["task"]

Note that above in the reindex command, we set `wait_for_completion` to false. Inference can possibly take a while and we might run the risk of our command timing out.
The call above will return a task that we can watch and see its progress the the `tasks` endpoint:

In [ ]:
task_response = elasticsearch.tasks.get(task_id=task_id)
print(json.dumps(task_response.body, indent=2))

# Query the new Elasticsearch index

We will exemplify:

- [how to replicate App Search queries](#how-to-build-app-search-like-queries)
- [how to do semantic search using ELSER](#how-to-do-semantic-search-using-elser)
- [how to combine App Search queries and ELSER](#how-to-combine-app-search-queries-with-elser)

### How to build App Search like queries

App Search exposes a [search_explain API](https://www.elastic.co/guide/en/app-search/current/search-explain.html) that receives an App Search query and returns the Elasticsearch query built by App Search.

```bash
curl -X POST '${ENTERPRISE_SEARCH_BASE_URL}/api/as/v1/engines/national-parks-demo/search_explain' \
-H 'Content-Type: application/json' \
-H 'Authorization: Bearer private-xxxxxx' \
-d '{
  "query": "park"
}'
```

From the output of the API call above, we can see the actual Elasticsearch query that will be used. Below, we are using this query as a base to build our own App Search like query using query rules and our Elasticsearch synonyms. The query is further enhanced by augmentation with the built-in App Search multifield types for such things as stemming and prefix matching.

In [ ]:
QUERY_STRING = "park"

result_fields = list(schema.keys())

text_fields = [field_name for field_name in schema if schema[field_name] == "text"]
best_fields = [field_name + ".stem" for field_name in text_fields]

cross_fields = []

for text_field in text_fields:
    cross_fields.append(text_field + "^1.0")
    cross_fields.append(text_field + ".stem^0.95")
    cross_fields.append(text_field + ".prefix^0.1")
    cross_fields.append(text_field + ".joined^0.75")
    cross_fields.append(text_field + ".delimiter^0.4")

app_search_query_payload = {
    "query": {
        "rule_query": {
            "organic": {
                "bool": {
                    "should": [
                        {
                            "multi_match": {
                                "query": QUERY_STRING,
                                "minimum_should_match": "1<-1 3<49%",
                                "type": "cross_fields",
                                "fields": cross_fields,
                            }
                        },
                        {
                            "multi_match": {
                                "query": QUERY_STRING,
                                "minimum_should_match": "1<-1 3<49%",
                                "type": "best_fields",
                                "fuzziness": "AUTO",
                                "prefix_length": 2,
                                "fields": best_fields,
                            }
                        },
                    ]
                }
            },
            "ruleset_id": ENGINE_NAME,
            "match_criteria": {"user_query": QUERY_STRING},
        }
    },
    "sort": [{"_score": "desc"}, {"_doc": "desc"}],
    "highlight": {
        "fragment_size": 300,
        "type": "plain",
        "number_of_fragments": 1,
        "order": "score",
        "encoder": "html",
        "require_field_match": False,
        "fields": {},
    },
    "size": 10,
    "_source": result_fields,
}

print(f"Elasticsearch payload:\n{json.dumps(app_search_query_payload, indent=2)}")

Now that we have our fully flushed out query, we can use that to perform the actual search:

In [ ]:
results = elasticsearch.search(
    index=SOURCE_INDEX,
    query=app_search_query_payload["query"],
    highlight=app_search_query_payload["highlight"],
    source=app_search_query_payload["_source"],
    sort=app_search_query_payload["sort"],
    size=app_search_query_payload["size"],
)
print(json.dumps(results.body, indent=2))

### How to do semantic search using ELSER

If you [enabled and reindexed your data with ELSER](#add-sparse_vector-fields-for-semantic-search-optional), we can now use this to do semantic search.
For each `spare_vector` we will generate a `text_expansion` query. These `text_expansion` queries will be added as `should` clauses to a top-level `bool` query.
We also use `min_score` because we want to exclude less relevant results. 

In [ ]:
# replace with your own
QUERY_STRING = "Which national park has dangerous wild animals?"
text_expansion_queries = []

for field_name in SPARSE_VECTOR_FIELDS:
    text_expansion_queries.append(
        {
            "text_expansion": {
                field_name: {"model_id": ".elser_model_2", "model_text": QUERY_STRING}
            }
        }
    )

semantic_query = {"bool": {"should": text_expansion_queries}}
print(f"Elasticsearch query:\n{json.dumps(semantic_query, indent=2)}\n")

In [ ]:
results = elasticsearch.search(index=DEST_INDEX, query=semantic_query, min_score=20)
print(f"Query results:\n{json.dumps(results.body, indent=2)}\n")

### How to combine App Search queries with ELSER

We will now provide an example on how to combine the previous two queries into a single query that applies both BM25 search and semantic search.
In the previous examples, we have a `bool` query with `should` clauses.
We will combine them in a single `bool` query and wrap this `bool` query in a `rule_query`.
The `rule_query` is used to pin results based on the query string, similarly to App Search curations.
The high-level structure of the query is following:

```json
GET [DEST-INDEX]
{
  "query": {
    "rule_query": {
      "organic": {
        "bool": {
          "should": [
            // multi_match query with best_fields from App Search generated query
            // multi_match query with cross_fields from App Search generated query
            // text_expansion queries for sparse_vector fields
          ]
        }
      }  
    }
  }
}
```

We are again using `min_score` to exclude less relevant results.
In our example we are not boosting any of the `should` clauses, but this can be a way to boost ELSER results over BM25 results.

In [ ]:
payload = app_search_query_payload.copy()

for text_expansion_query in text_expansion_queries:
    payload["query"]["rule_query"]["organic"]["bool"]["should"].append(
        text_expansion_query
    )

print(f"Elasticsearch payload:\n{json.dumps(payload, indent=2)}\n")

In [ ]:
results = elasticsearch.search(
    index=SOURCE_INDEX,
    query=payload["query"],
    highlight=payload["highlight"],
    source=payload["_source"],
    sort=payload["sort"],
    size=payload["size"],
    min_score=1,
)

print(f"Text expansion query results:\n{json.dumps(results.body, indent=2)}\n")